In [120]:
# Equipment setup

import pyvisa
import time
import subprocess
import serial

# Establish connection to the power supply
rm = pyvisa.ResourceManager("@py")

#psu = rm.open_resource('TCPIP::169.254.58.10::gpib0,7::INSTR')                  ## for the Agilent power supply

psu = rm.open_resource('TCPIP::169.254.201.77::lan0::INSTR')                    ## for the Keysight power supply

clk = rm.open_resource('TCPIP::169.254.201.101::lan0::INSTR')

# Need to verify that this is the power supply. psu_verification should be 
# something like "Keysight Technologies,E3646A,0,X.X-Y.Y-Z.Z"
psu_verification = psu.query("*IDN?")
clk_verification = clk.query("*IDN?")

print("Power supply:", psu_verification)
print("Clock generator:", clk_verification)

# psu.write("*RST")  # Reset the power supply

start_voltage = 0.55

end_voltage = 0.65

step_size = 0.05  

start_clock = 20

end_clock = 40

step_size = 20

# Loop through the voltage range and set the voltage
current_voltage = start_voltage 

current_clock = start_clock

psu.write("OUTP ON, (@2)")

# Only channel 2 works
psu.write("INST:SEL CH2")

# Set current to 100mA
psu.write("CURR 0.1")

# Start the clock at 20MHz
clk.write(":OUTP1:POS ON") # Initialize clk gen channel 1 output
clk.write(":FUNC:PATT")
clk.write(":FREQ 20MHz")

Power supply: Keysight Technologies,E36312A,MY57010726,1.0.4-1.0.0-1.04

Clock generator: Agilent Technologies, 81134A, DE42800131, Ver 2.9.3



10

In [122]:
while current_clock <= end_clock:


     while current_voltage <= end_voltage: # Set the voltage   
          psu.write(f"VOLT {current_voltage}")
          psu.write("OUTP ON, (@2)")

          voltage = psu.query("VOLT?") 
          print(f"**Voltage set to: {voltage}V**") 

          measured_voltages = []
          measured_currents = []

          # remake to adjust with the frequency
          # subprocess.run(["make", "clean"], cwd="../../")

          # with open("../../platform/bearly24/freq.h", "w") as f:
          #      f.write('''#ifndef __FREQ_H
          #                #define __FREQ_H
          #                ''')
          #      f.write(f'''#define MTIME_FREQ     {freq}
          #                #define SYS_CLK_FREQ   {freq}
          #                ''')
          #      f.write("#endif")

          # subprocess.run(["make", "shmoo_bearly"], cwd="../../")

          # test subprocess with simple hello
          with subprocess.Popen(["uart_tsi", "+tty=/dev/ttyUSB1", "+baudrate=921600", "../../build/shmoo_bench.elf"]) as proc:
               with serial.Serial('/dev/ttyUSB0', 115200) as ser:
                    while (ser.inWaiting() == 0):
                         ## measure the voltage + current
                         measured_voltages.append(psu.query("MEAS:VOLT?"))
                         measured_currents.append(psu.query("MEAS:CURR?"))
                         # print("hi")
                    line = ser.readline()
                    new_line = line.decode("utf-8")

                    # stop_event.set()
                    # power_thread.join()
                    # total_power = power_queue.get()

                    with open('../../output.txt', 'a') as file:
                         # file.write(f"{new_line} {total_power:.2f}\n")
                         file.write(f"{new_line}")
                         proc.terminate()
                         proc.wait()
          print("Done: now reset during the sleep before next iteration")
          time.sleep(7)

          current_voltage += step_size

print("Finished")

psu.close()

**Voltage set to: +5.50000000E-01
V**


Starting UART-based TSI
Usage: ./uart_tsi +tty=/dev/pts/xx <PLUSARGS> <bin>
       ./uart_tsi +tty=/dev/ttyxx  <PLUSARGS> <bin>
       ./uart_tsi +tty=/dev/ttyxx  +no_hart0_msip +init_write=0x80000000:0xdeadbeef none
       ./uart_tsi +tty=/dev/ttyxx  +no_hart0_msip +init_read=0x80000000 none
       ./uart_tsi +tty=/dev/ttyxx  +selfcheck <bin>
       ./uart_tsi +tty=/dev/ttyxx  +baudrate=921600 <bin>
Attempting to open TTY at /dev/ttyUSB1
Checking connection status with /dev/ttyUSB1
Connection succeeded
Done, shutting down, flushing UART
Done: now reset during the sleep before next iteration
**Voltage set to: +6.00000000E-01
V**


Starting UART-based TSI
Usage: ./uart_tsi +tty=/dev/pts/xx <PLUSARGS> <bin>
       ./uart_tsi +tty=/dev/ttyxx  <PLUSARGS> <bin>
       ./uart_tsi +tty=/dev/ttyxx  +no_hart0_msip +init_write=0x80000000:0xdeadbeef none
       ./uart_tsi +tty=/dev/ttyxx  +no_hart0_msip +init_read=0x80000000 none
       ./uart_tsi +tty=/dev/ttyxx  +selfcheck <bin>
       ./uart_tsi +tty=/dev/ttyxx  +baudrate=921600 <bin>
Attempting to open TTY at /dev/ttyUSB1
Checking connection status with /dev/ttyUSB1
Connection succeeded
Done, shutting down, flushing UART
Done: now reset during the sleep before next iteration


KeyboardInterrupt: 

In [123]:
print(measured_voltages)
print(measured_currents)

['+5.99905000E-01\n', '+5.99905000E-01\n', '+5.99070000E-01\n']
['+1.17490000E-02\n', '+1.17490000E-02\n', '+1.17680000E-02\n']


In [186]:
import pyvisa
import time
import subprocess

# Establish connection to the power supply
rm = pyvisa.ResourceManager("@py")

psu = rm.open_resource('TCPIP::169.254.201.77::lan0::INSTR')
psu.write("INST:SEL CH2")
psu.write(f"VOLT 1.2")
psu.write("CURR 0.1")
psu.write("OUTP ON, (@2)")
psu.query("VOLT?") 


'+1.20000000E+00\n'

In [10]:
import subprocess

freq = 16000000

subprocess.run(["make", "clean"], cwd="../../")

with open("../../platform/bearly24/freq.h", "w") as f:
    f.write('''#ifndef __FREQ_H
            #define __FREQ_H
            ''')
    f.write(f'''#define MTIME_FREQ     {freq}
            #define SYS_CLK_FREQ   {freq}
            ''')
    f.write("#endif")

subprocess.run(["make", "shmoo_bearly"], cwd="../../")


cmake -S ./ -B ./build/ -D CMAKE_TOOLCHAIN_FILE=./riscv-gcc.cmake -DCHIP=bearly24
-- Including HTIF to target
-- Including CLINT to target
-- ================ Glossy config ================
--  Building for CHIP: "bearly24"
--  Reading config from "/tools/C/tedkim7/sp24-Baremetal-IDE/platform/bearly24/CMakeLists.txt"
--  Terminal Device: UART0
--  Linker Script: /tools/C/tedkim7/sp24-Baremetal-IDE/platform/bearly24/bearly24.ld
-- ===============================================
-- Configuring done (0.1s)
-- Generating done (1.5s)
-- Build files have been written to: /tools/C/tedkim7/sp24-Baremetal-IDE/build
cmake --build ./build/ --target shmoo_bench
gmake[1]: Entering directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
gmake[2]: Entering directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
gmake[3]: Entering directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
gmake[4]: Entering directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
gmake[4]: Leaving directory '/bwrcq/C/tedkim7/sp24

/tools/C/tedkim7/sp24-Baremetal-IDE/glossy/src/sys/exit.c: In function '_exit':
/tools/C/tedkim7/sp24-Baremetal-IDE/glossy/src/sys/exit.c:4:48: warning: unused parameter 'code' [-Wunused-parameter]
    4 | __attribute__((weak, noreturn)) void _exit(int code) {
      |                                            ~~~~^~~~


[ 50%] Building C object glossy/CMakeFiles/glossy.dir/src/sys/time.c.obj
[ 52%] Building C object glossy/CMakeFiles/glossy.dir/src/sys/write.c.obj


/tools/C/tedkim7/sp24-Baremetal-IDE/glossy/src/sys/write.c: In function '_write':
/tools/C/tedkim7/sp24-Baremetal-IDE/glossy/src/sys/write.c:8:42: warning: unused parameter 'fd' [-Wunused-parameter]
    8 | __attribute__((weak)) ssize_t _write(int fd, const void *ptr, size_t len) {
      |                                      ~~~~^~


[ 55%] Linking C static library libglossy.a
gmake[4]: Leaving directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
[ 94%] Built target glossy
gmake[4]: Entering directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
gmake[4]: Leaving directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
gmake[4]: Entering directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
[ 97%] Building C object CMakeFiles/shmoo_bench.dir/shmoo_bench/pll_bench/src/main.c.obj


/tools/C/tedkim7/sp24-Baremetal-IDE/shmoo_bench/pll_bench/src/main.c: In function 'app_main':
/tools/C/tedkim7/sp24-Baremetal-IDE/shmoo_bench/pll_bench/src/main.c:116:5: warning: implicit declaration of function 'reg_write8' [-Wimplicit-function-declaration]
  116 |     reg_write8(0x130000, 0x1);
      |     ^~~~~~~~~~
/tools/C/tedkim7/sp24-Baremetal-IDE/shmoo_bench/pll_bench/src/main.c: In function 'main':
/tools/C/tedkim7/sp24-Baremetal-IDE/shmoo_bench/pll_bench/src/main.c:126:14: warning: unused parameter 'argc' [-Wunused-parameter]
  126 | int main(int argc, char **argv) {
      |          ~~~~^~~~
/tools/C/tedkim7/sp24-Baremetal-IDE/shmoo_bench/pll_bench/src/main.c:126:27: warning: unused parameter 'argv' [-Wunused-parameter]
  126 | int main(int argc, char **argv) {
      |                    ~~~~~~~^~~~


[100%] Linking C executable shmoo_bench.elf
gmake[4]: Leaving directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
[100%] Built target shmoo_bench
gmake[3]: Leaving directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
gmake[2]: Leaving directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'
gmake[1]: Leaving directory '/bwrcq/C/tedkim7/sp24-Baremetal-IDE/build'


/bwrcq/C/ee290-fa24-2/.conda-env/riscv-tools/bin/../lib/gcc/riscv64-unknown-elf/13.2.0/../../../../riscv64-unknown-elf/bin/ld: warning: shmoo_bench.elf has a LOAD segment with RWX permissions


CompletedProcess(args=['make', 'shmoo_bearly'], returncode=0)